In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection
import logging
import requests
load_dotenv(override=True)
openai = OpenAI()
MODEL = 'gpt-5-mini'

In [2]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 3/3 [00:39<00:00, 13.33s/it]


In [3]:
len(deals)

30

In [4]:
deals[10].describe()

"Title: Refurb Logitech G815 Lightsync RGB Mechanical Gaming Keyboard for $70 + free shipping\nDetails: That's $60 less than you'd pay for one in factory sealed condition. A one-year Allstate warranty is provided with purchase. Buy Now at eBay\nFeatures: \nURL: https://www.dealnews.com/products/Logitech/Logitech-G815-Lightsync-RGB-Mechanical-Gaming-Keyboard/497140.html?iref=rss-c39"

### We are going to ask GPT-5-mini to summarize deals and identify their price

In [5]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [6]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [7]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Best Buy Outlet Event: Up to 50% off + free shipping
Details: Take up to half off clearance, open-box, and refurbished items. Many items will ship for free and some may be available for pickup as well. Shop Now at Best Buy
Features: 
URL: https://www.dealnews.com/Best-Buy-Outlet-Event-Up-to-50-off-free-shipping/21800020.html?iref=rss-c142

Title: Refurb Unlocked Samsung Galaxy 

In [8]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="minimal")
results = response.choices[0].message.parsed
results

DealSelection(deals=[Deal(product_description='Samsung Galaxy S23+ (refurbished, unlocked) with 256 GB storage. The phone features a 6.6-inch Dynamic AMOLED 2X Infinity-O FHD+ display, Android 13, and a large 4,700 mAh battery. This model includes typical S23+ flagship hardware and is sold with a one-year Allstate warranty from a Top Rated Plus eBay seller.', price=299.0, url='https://www.dealnews.com/products/Samsung/Unlocked-Samsung-Galaxy-S23-256-GB-Smartphone/417747.html?iref=rss-c142'), Deal(product_description='Apple Watch Series 10 (46mm, GPS) refurbished smartwatch from 2024. It has an always-on OLED Retina display, up to 36-hour battery life in low power mode, heart rate monitoring, cycle and activity tracking, GPS, fall and crash detection, voice control, and Bluetooth/Wi‑Fi connectivity. The listing notes the S10 SiP chipset and includes a one-year Allstate warranty.', price=270.0, url='https://www.dealnews.com/products/Apple/Apple-Watch-Series-10-46-mm-GPS-Smartwatch-2024/4

In [9]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


Samsung Galaxy S23+ (refurbished, unlocked) with 256 GB storage. The phone features a 6.6-inch Dynamic AMOLED 2X Infinity-O FHD+ display, Android 13, and a large 4,700 mAh battery. This model includes typical S23+ flagship hardware and is sold with a one-year Allstate warranty from a Top Rated Plus eBay seller.
299.0
https://www.dealnews.com/products/Samsung/Unlocked-Samsung-Galaxy-S23-256-GB-Smartphone/417747.html?iref=rss-c142

Apple Watch Series 10 (46mm, GPS) refurbished smartwatch from 2024. It has an always-on OLED Retina display, up to 36-hour battery life in low power mode, heart rate monitoring, cycle and activity tracking, GPS, fall and crash detection, voice control, and Bluetooth/Wi‑Fi connectivity. The listing notes the S10 SiP chipset and includes a one-year Allstate warranty.
270.0
https://www.dealnews.com/products/Apple/Apple-Watch-Series-10-46-mm-GPS-Smartwatch-2024/485220.html?iref=rss-c142

Onn 75-inch 4K LED UHD Roku Smart TV (model 100044717). This large-format TV 

In [10]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [11]:
from agents.scanner_agent import ScannerAgent

In [12]:
agent = ScannerAgent()
result = agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 30 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


In [13]:
result

DealSelection(deals=[Deal(product_description='Refurbished Samsung Galaxy S23+ unlocked smartphone with 256GB storage. It runs Android 13 on a 6.6" Dynamic AMOLED 2X Infinity-O FHD+ display, and is powered by a 4,700 mAh battery. The listing notes model SM-S916UZKAXAA and includes a one-year Allstate warranty from a Top Rated Plus eBay seller.', price=299.0, url='https://www.dealnews.com/products/Samsung/Unlocked-Samsung-Galaxy-S23-256-GB-Smartphone/417747.html?iref=rss-c142'), Deal(product_description='Refurbished Apple Watch Series 10 GPS, 46mm case (2024). Features include an always-on Retina OLED display, heart rate monitoring, cycle and activity tracking, GPS, voice control, fall and crash detection, and up to about 36 hours in low power mode. The smartwatch uses the S10 SiP and supports wireless and Bluetooth connectivity.', price=270.0, url='https://www.dealnews.com/products/Apple/Apple-Watch-Series-10-46-mm-GPS-Smartwatch-2024/485220.html?iref=rss-c142'), Deal(product_descripti

In [14]:
load_dotenv(override=True)

True

In [15]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [16]:
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [17]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [18]:
push("MASSIVE DEAL!!")

Push: MASSIVE DEAL!!


In [19]:
from agents.messaging_agent import MessagingAgent

agent = MessagingAgent()
agent.push("SUCH A MASSIVE DEAL!!")

INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has initialized Pushover and Claude
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification


In [20]:
agent.notify("A special deal on Sumsung 60 inch LED TV going at a great bargain", 300, 1000, "www.samsung.com")

INFO:root:[Messaging Agent] Messaging Agent is using Claude to craft the message
08:01:12 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
08:01:15 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification
INFO:root:[Messaging Agent] Messaging Agent has completed
